In [1]:
import requests

# URL endpoint do pobrania wszystkich stacji
url = "https://api.gios.gov.pl/pjp-api/rest/station/sensors/20201"

try:
    response = requests.get(url)
    response.raise_for_status()  

    station = response.json()  

    print(station)

except requests.RequestException as e:
    print(f"Błąd podczas połączenia z API: {e}")


[{'id': 31415, 'stationId': 20201, 'param': {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}}, {'id': 31416, 'stationId': 20201, 'param': {'paramName': 'pył zawieszony PM2.5', 'paramFormula': 'PM2.5', 'paramCode': 'PM2.5', 'idParam': 69}}]


In [4]:

import requests


station_id = 20201
sensors_url = f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}"

try:
    sensors_response = requests.get(sensors_url)
    sensors_response.raise_for_status()
    sensors = sensors_response.json()


    pm10_sensor = next((s for s in sensors if s['param']['paramCode'] == 'PM10'), None)

    if not pm10_sensor:
        print("Nie znaleziono czujnika PM10 dla tej stacji.")
    else:
        sensor_id = pm10_sensor['id']

        data_url = f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_id}"
        data_response = requests.get(data_url)
        data_response.raise_for_status()
        data = data_response.json()


        print(f"Pomiar PM10 dla stacji ID {station_id}:")
        for entry in data['values']:
            if entry['value'] is not None:
                print(f"{entry['date']}: {entry['value']} µg/m³")

except requests.RequestException as e:
    print(f"Błąd podczas komunikacji z API: {e}")


Pomiar PM10 dla stacji ID 20201:
2025-05-23 16:00:00: 11.7 µg/m³
2025-05-23 15:00:00: 14.0 µg/m³
2025-05-23 14:00:00: 15.0 µg/m³
2025-05-23 13:00:00: 15.1 µg/m³
2025-05-23 12:00:00: 15.0 µg/m³
2025-05-23 11:00:00: 14.1 µg/m³
2025-05-23 10:00:00: 14.2 µg/m³
2025-05-23 09:00:00: 12.9 µg/m³
2025-05-23 08:00:00: 14.9 µg/m³
2025-05-23 07:00:00: 15.7 µg/m³
2025-05-23 06:00:00: 12.2 µg/m³
2025-05-23 05:00:00: 11.2 µg/m³
2025-05-23 04:00:00: 11.6 µg/m³
2025-05-23 03:00:00: 13.6 µg/m³
2025-05-23 02:00:00: 15.4 µg/m³
2025-05-23 01:00:00: 16.0 µg/m³
2025-05-23 00:00:00: 16.8 µg/m³
2025-05-22 23:00:00: 16.4 µg/m³
2025-05-22 22:00:00: 12.5 µg/m³
2025-05-22 21:00:00: 13.2 µg/m³
2025-05-22 20:00:00: 44.1 µg/m³
2025-05-22 19:00:00: 34.5 µg/m³
2025-05-22 18:00:00: 24.9 µg/m³
2025-05-22 17:00:00: 47.1 µg/m³
2025-05-22 16:00:00: 37.2 µg/m³
2025-05-22 15:00:00: 78.2 µg/m³
2025-05-22 14:00:00: 45.7 µg/m³
2025-05-22 13:00:00: 10.5 µg/m³
2025-05-22 12:00:00: 34.2 µg/m³
2025-05-22 11:00:00: 41.3 µg/m³
2025-05

In [5]:
from kafka import KafkaProducer
import json
import requests
import time


producer = KafkaProducer(
    bootstrap_servers='broker:9092', 
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

station_id = 20201
sensors_url = f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}"

try:
    sensors_response = requests.get(sensors_url)
    sensors_response.raise_for_status()
    sensors = sensors_response.json()

    pm10_sensor = next((s for s in sensors if s['param']['paramCode'] == 'PM10'), None)

    if not pm10_sensor:
        print("Nie znaleziono czujnika PM10 dla tej stacji.")
    else:
        sensor_id = pm10_sensor['id']
        data_url = f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_id}"
        data_response = requests.get(data_url)
        data_response.raise_for_status()
        data = data_response.json()

        print(f"Wysyłam dane PM10 do Kafka dla stacji ID {station_id}:")
        for entry in data['values']:
            if entry['value'] is not None:
                msg = {
                    'station_id': station_id,
                    'datetime': entry['date'],
                    'value': entry['value'],
                    'unit': 'µg/m³'
                }
                print(f"Wysyłanie: {msg}")
                producer.send('pm10_readings', msg)
                time.sleep(0.5)  # mały delay

        producer.flush()

except requests.RequestException as e:
    print(f"Błąd podczas komunikacji z API: {e}")
except Exception as e:
    print(f"Inny błąd: {e}")


Wysyłam dane PM10 do Kafka dla stacji ID 20201:
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 17:00:00', 'value': 9.9, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 16:00:00', 'value': 11.7, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 15:00:00', 'value': 14.0, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 14:00:00', 'value': 15.0, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 13:00:00', 'value': 15.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 12:00:00', 'value': 15.0, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 11:00:00', 'value': 14.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 10:00:00', 'value': 14.2, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-23 09:00:00', 'value': 12.9, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05

In [8]:
from kafka import KafkaProducer
import json
import requests
import time

producer = KafkaProducer(
    bootstrap_servers='broker:9092', 
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

station_id = 20201
sensors_url = f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}"

try:
    sensors_response = requests.get(sensors_url)
    sensors_response.raise_for_status()
    sensors = sensors_response.json()

    pm10_sensor = next((s for s in sensors if s['param']['paramCode'] == 'PM10'), None)

    if not pm10_sensor:
        print("Nie znaleziono czujnika PM10 dla tej stacji.")
    else:
        sensor_id = pm10_sensor['id']
        data_url = f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_id}"
        data_response = requests.get(data_url)
        data_response.raise_for_status()
        data = data_response.json()

        print(f"Wysyłam dane PM10 do Kafka dla stacji ID {station_id}:")
        for entry in reversed(data['values']):
            if entry['value'] is not None:
                msg = {
                    'station_id': station_id,
                    'datetime': entry['date'],
                    'value': entry['value'],
                    'unit': 'µg/m³'
                }
                print(f"Wysyłanie: {msg}")
                producer.send('pm10_readings', msg)
                time.sleep(10)  # mały delay

        producer.flush()

except requests.RequestException as e:
    print(f"Błąd podczas komunikacji z API: {e}")
except Exception as e:
    print(f"Inny błąd: {e}")


Wysyłam dane PM10 do Kafka dla stacji ID 20201:
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 02:00:00', 'value': 12.4, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 03:00:00', 'value': 12.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 04:00:00', 'value': 12.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 05:00:00', 'value': 12.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 06:00:00', 'value': 11.7, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 07:00:00', 'value': 13.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 08:00:00', 'value': 13.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 09:00:00', 'value': 14.5, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 10:00:00', 'value': 13.0, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-0

In [9]:
from kafka import KafkaProducer
import json
import requests
import time

producer = KafkaProducer(
    bootstrap_servers='broker:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

station_id = 20201
sensors_url = f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}"

# Pamiętaj wysłane daty
sent_timestamps = set()

def fetch_pm10_sensor_id():
    sensors_response = requests.get(sensors_url)
    sensors_response.raise_for_status()
    sensors = sensors_response.json()
    pm10_sensor = next((s for s in sensors if s['param']['paramCode'] == 'PM10'), None)
    return pm10_sensor['id'] if pm10_sensor else None

def fetch_pm10_data(sensor_id):
    data_url = f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_id}"
    data_response = requests.get(data_url)
    data_response.raise_for_status()
    return data_response.json()

try:
    sensor_id = fetch_pm10_sensor_id()
    if not sensor_id:
        print("Nie znaleziono czujnika PM10.")
    else:
        while True:
            data = fetch_pm10_data(sensor_id)
            new_entries = []

            for entry in reversed(data['values']):
                timestamp = entry['date']
                value = entry['value']

                if value is not None and timestamp not in sent_timestamps:
                    msg = {
                        'station_id': station_id,
                        'datetime': timestamp,
                        'value': value,
                        'unit': 'µg/m³'
                    }
                    new_entries.append(msg)
                    sent_timestamps.add(timestamp)

            if new_entries:
                print(f"Wysyłam {len(new_entries)} nowych wiadomości do Kafka:")
                for msg in new_entries:
                    print(f"Wysyłanie: {msg}")
                    producer.send('pm10_readings', msg)
                    time.sleep(10)

                producer.flush()
            else:
                print("Brak nowych danych.")

            time.sleep(20)  # sprawdzaj co 10 minut

except requests.RequestException as e:
    print(f"Błąd API: {e}")
except Exception as e:
    print(f"Inny błąd: {e}")

Wysyłam 65 nowych wiadomości do Kafka:
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 02:00:00', 'value': 12.4, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 03:00:00', 'value': 12.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 04:00:00', 'value': 12.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 05:00:00', 'value': 12.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 06:00:00', 'value': 11.7, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 07:00:00', 'value': 13.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 08:00:00', 'value': 13.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 09:00:00', 'value': 14.5, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 10:00:00', 'value': 13.0, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 11:0

KeyboardInterrupt: 

In [10]:
from kafka import KafkaProducer
import json
import requests
import time

producer = KafkaProducer(
    bootstrap_servers='broker:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

station_id = 20201
sensors_url = f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}"

# Pamiętaj wysłane daty
sent_timestamps = set()

def fetch_pm10_sensor_id():
    sensors_response = requests.get(sensors_url)
    sensors_response.raise_for_status()
    sensors = sensors_response.json()
    pm10_sensor = next((s for s in sensors if s['param']['paramCode'] == 'PM10'), None)
    return pm10_sensor['id'] if pm10_sensor else None

def fetch_pm10_data(sensor_id):
    data_url = f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_id}"
    data_response = requests.get(data_url)
    data_response.raise_for_status()
    return data_response.json()

try:
    sensor_id = fetch_pm10_sensor_id()
    if not sensor_id:
        print("Nie znaleziono czujnika PM10.")
    else:
        while True:
            data = fetch_pm10_data(sensor_id)
            new_entries = []

            for entry in reversed(data['values']):
                timestamp = entry['date']
                value = entry['value']

                if value is not None and timestamp not in sent_timestamps:
                    msg = {
                        'station_id': station_id,
                        'datetime': timestamp,
                        'value': value,
                        'unit': 'µg/m³'
                    }
                    new_entries.append(msg)
                    sent_timestamps.add(timestamp)

            if new_entries:
                print(f"Wysyłam {len(new_entries)} nowych wiadomości do Kafka:")
                for msg in new_entries:
                    print(f"Wysyłanie: {msg}")
                    producer.send('pm10_readings', msg)
                    time.sleep(10)

                producer.flush()
            else:
                print("Brak nowych danych.")

            time.sleep(20)  # sprawdzaj co 10 minut

except requests.RequestException as e:
    print(f"Błąd API: {e}")
except Exception as e:
    print(f"Inny błąd: {e}")

Wysyłam 66 nowych wiadomości do Kafka:
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 02:00:00', 'value': 12.4, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 03:00:00', 'value': 12.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 04:00:00', 'value': 12.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 05:00:00', 'value': 12.1, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 06:00:00', 'value': 11.7, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 07:00:00', 'value': 13.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 08:00:00', 'value': 13.8, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 09:00:00', 'value': 14.5, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 10:00:00', 'value': 13.0, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'datetime': '2025-05-21 11:0

In [18]:
from kafka import KafkaProducer
import json
import requests
import time
from datetime import datetime

producer = KafkaProducer(
    bootstrap_servers='broker:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

station_id = 20201
sensors_url = f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}"

sent_timestamps = set()
reading_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def fetch_pm10_sensor_id():
    sensors_response = requests.get(sensors_url)
    sensors_response.raise_for_status()
    sensors = sensors_response.json()
    pm10_sensor = next((s for s in sensors if s['param']['paramCode'] == 'PM10'), None)
    return pm10_sensor['id'] if pm10_sensor else None

def fetch_pm10_data(sensor_id):
    data_url = f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_id}"
    data_response = requests.get(data_url)
    data_response.raise_for_status()
    return data_response.json()

try:
    sensor_id = fetch_pm10_sensor_id()
    if not sensor_id:
        print("Nie znaleziono czujnika PM10.")
    else:
        while True:
            data = fetch_pm10_data(sensor_id)
            new_entries = []

            for entry in reversed(data['values']):
                timestamp = entry['date']
                value = entry['value']

                if value is not None and timestamp not in sent_timestamps:
                    msg = {
                        'station_id': station_id,
                        'reading_date': reading_date,
                        'datetime_from_sensor': timestamp,
                        'value': value,
                        'unit': 'µg/m³'
                    }
                    new_entries.append(msg)
                    sent_timestamps.add(timestamp)

            if new_entries:
                print(f"Wysyłam {len(new_entries)} nowych wiadomości do Kafka:")
                for msg in new_entries:
                    print(f"Wysyłanie: {msg}")
                    producer.send('pm10_readings', msg)
                    time.sleep(10)

                producer.flush()
            else:
                print("Brak nowych danych.")

            time.sleep(900)  # sprawdzaj co 15 minut

except requests.RequestException as e:
    print(f"Błąd API: {e}")
except Exception as e:
    print(f"Inny błąd: {e}")

Wysyłam 59 nowych wiadomości do Kafka:
Wysyłanie: {'station_id': 20201, 'reading_date': '2025-05-25 10:38:36', 'datetime_from_sensor': '2025-05-23 02:00:00', 'value': 15.4, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'reading_date': '2025-05-25 10:38:36', 'datetime_from_sensor': '2025-05-23 03:00:00', 'value': 13.6, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'reading_date': '2025-05-25 10:38:36', 'datetime_from_sensor': '2025-05-23 04:00:00', 'value': 11.6, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'reading_date': '2025-05-25 10:38:36', 'datetime_from_sensor': '2025-05-23 05:00:00', 'value': 11.2, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'reading_date': '2025-05-25 10:38:36', 'datetime_from_sensor': '2025-05-23 06:00:00', 'value': 12.2, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'reading_date': '2025-05-25 10:38:36', 'datetime_from_sensor': '2025-05-23 07:00:00', 'value': 15.7, 'unit': 'µg/m³'}
Wysyłanie: {'station_id': 20201, 'reading_date': '2025-

KeyboardInterrupt: 